In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

pip install 'aif360[LawSchoolGPA]'


In [2]:
raw_df=pd.read_csv("data.csv")
raw_df.drop_duplicates()
display(raw_df)

type_school school_accreditation  gender         interest residence  \
0      Academic                    A    Male  Less Interested     Urban   
1      Academic                    A    Male  Less Interested     Urban   
2      Academic                    B  Female  Very Interested     Urban   
3    Vocational                    B    Male  Very Interested     Rural   
4      Academic                    A  Female  Very Interested     Urban   
..          ...                  ...     ...              ...       ...   
995  Vocational                    A  Female  Very Interested     Rural   
996    Academic                    B  Female  Less Interested     Rural   
997  Vocational                    A    Male  Less Interested     Urban   
998    Academic                    B    Male        Uncertain     Rural   
999    Academic                    B    Male       Interested     Urban   

     parent_age  parent_salary  house_area  average_grades  \
0            56        6950000        83.0           84.09   
1            57        4410000        76.8           86.91   
2            50        6500000        80.6           87.43   
3            49        6600000        78.2           82.12   
4            57        5250000        75.1           86.79   
..          ...            ...         ...             ...   
995          49        7420000        63.6           85.99   
996          51        7480000        84.3           89.72   
997          49        5550000        75.2           79.56   
998          53        5840000       105.8           87.18   
999          50        2940000        69.1           86.13   

     parent_was_in_college  will_go_to_college  
0                    False                True  
1                    False                True  
2                    False                True  
3                     True                True  
4                    False               False  
..                     ...                 ...  
995                   True                True  
996                   True                True  
997                  False                True  
998                   True                True  
999                   True               False  

[1000 rows x 11 columns]

In [3]:
man_df=raw_df.copy().loc[raw_df["gender"]=="Male"].loc[:,['gender','will_go_to_college']]
display(man_df)
woman_df=raw_df.copy().loc[raw_df["gender"]!="Male"].loc[:,['gender','will_go_to_college']]
display(woman_df)

gender  will_go_to_college
0     Male                True
1     Male                True
3     Male                True
6     Male                True
7     Male               False
..     ...                 ...
989   Male               False
993   Male               False
997   Male                True
998   Male                True
999   Male               False

[515 rows x 2 columns]

gender  will_go_to_college
2    Female                True
4    Female               False
5    Female               False
8    Female                True
9    Female               False
..      ...                 ...
991  Female                True
992  Female               False
994  Female               False
995  Female                True
996  Female                True

[485 rows x 2 columns]

In [4]:
print(man_df.shape[0]/raw_df.shape[0])
print(woman_df.shape[0]/raw_df.shape[0])

man_label=man_df["will_go_to_college"].value_counts()
print(man_label[True]/man_df.shape[0])
print(man_label[False]/man_df.shape[0])

women_label=woman_df["will_go_to_college"].value_counts()
print(women_label[True]/woman_df.shape[0])
print(women_label[False]/woman_df.shape[0])

0.515
0.485
0.48349514563106794
0.516504854368932
0.5175257731958763
0.4824742268041237


In [5]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

df=raw_df.copy(deep=True)
#normalize PreviousSalary feature
df["parent_age"]=min_max_normalisation(df["parent_age"])
df["parent_salary"]=min_max_normalisation(df["parent_salary"])
df["house_area"]=min_max_normalisation(df["house_area"])
df["average_grades"]=min_max_normalisation(df["average_grades"])

display(df)

type_school school_accreditation  gender         interest residence  \
0      Academic                    A    Male  Less Interested     Urban   
1      Academic                    A    Male  Less Interested     Urban   
2      Academic                    B  Female  Very Interested     Urban   
3    Vocational                    B    Male  Very Interested     Rural   
4      Academic                    A  Female  Very Interested     Urban   
..          ...                  ...     ...              ...       ...   
995  Vocational                    A  Female  Very Interested     Rural   
996    Academic                    B  Female  Less Interested     Rural   
997  Vocational                    A    Male  Less Interested     Urban   
998    Academic                    B    Male        Uncertain     Rural   
999    Academic                    B    Male       Interested     Urban   

     parent_age  parent_salary  house_area  average_grades  \
0          0.64       0.661111       0.630        0.395217   
1          0.68       0.378889       0.568        0.517826   
2          0.40       0.611111       0.606        0.540435   
3          0.36       0.622222       0.582        0.309565   
4          0.68       0.472222       0.551        0.512609   
..          ...            ...         ...             ...   
995        0.36       0.713333       0.436        0.477826   
996        0.44       0.720000       0.643        0.640000   
997        0.36       0.505556       0.552        0.198261   
998        0.52       0.537778       0.858        0.529565   
999        0.40       0.215556       0.491        0.483913   

     parent_was_in_college  will_go_to_college  
0                    False                True  
1                    False                True  
2                    False                True  
3                     True                True  
4                    False               False  
..                     ...                 ...  
995                   True                True  
996                   True                True  
997                  False                True  
998                   True                True  
999                   True               False  

[1000 rows x 11 columns]

In [6]:
folds=np.array_split(df.sample(frac=1),5)

In [7]:
def get_dataset(input_df):
    dataset=StandardDataset(df=input_df,label_name="will_go_to_college",favorable_classes=[True],scores_name=""
                        ,protected_attribute_names=["gender"],privileged_classes=[["Male"]]
                        ,categorical_features=['type_school',"school_accreditation","interest","residence","parent_was_in_college"])
    return dataset

In [41]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]
Accuracy_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateImpact_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateParity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualOpportunity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualizedOdds_data=pd.DataFrame(columns=Cross_Validation_Column)
#Read progress from local file if something happend that the jupyter kernel needs to be restarted (like fix bug from source library and reloaded it)
#merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"gender":1}]
unprivileged_groups = [{"gender":0}]

In [39]:
def cross_validation(row_name,train_test_process):
    global Accuracy_data
    global DisparateImpact_data
    global DisparateParity_data
    global EqualOpportunity_data
    global EqualizedOdds_data
    accuracy_row=[]
    disparateImpcat_row=[]
    disparateParity_row=[]
    EqualOpportunity_row=[]
    EqualizedOdss_row=[]
    for fold_index in range(5):
        test_fold=folds[fold_index]
        train_set=get_dataset(pd.concat([f for f in folds if f is not test_fold]))
        test_set=get_dataset(test_fold)
        test_pred=train_test_process(train_set,test_set)
        metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        #Record Result
        accuracy_row.append(metric.accuracy())
        disparateImpcat_row.append(metric.disparate_impact())
        disparateParity_row.append(metric.statistical_parity_difference())
        EqualOpportunity_row.append(metric.true_positive_rate_difference())
        EqualizedOdss_row.append(metric.false_positive_rate_difference())
    accuracy_row.append(np.average(accuracy_row))
    disparateImpcat_row.append(np.average(disparateImpcat_row))
    disparateParity_row.append(np.average(disparateParity_row))
    EqualOpportunity_row.append(np.average(EqualOpportunity_row))
    EqualizedOdss_row.append(np.average(EqualizedOdss_row))
    #Record Cross Validation
    Accuracy_data=pd.concat([Accuracy_data,pd.DataFrame([accuracy_row],index=[row_name],columns=Cross_Validation_Column)])
    Accuracy_data.columns.name="Accuracy"
    display(Accuracy_data)
    DisparateImpact_data=pd.concat([DisparateImpact_data,pd.DataFrame([disparateImpcat_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateImpact_data.columns.name="Disparate Impact"
    display(DisparateImpact_data)
    DisparateParity_data=pd.concat([DisparateParity_data,pd.DataFrame([disparateParity_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateParity_data.columns.name="Disparate Parity"
    display(DisparateParity_data)
    EqualOpportunity_data=pd.concat([EqualOpportunity_data,pd.DataFrame([EqualOpportunity_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualOpportunity_data.columns.name="Equal Opportuniy"
    display(EqualOpportunity_data)
    EqualizedOdds_data=pd.concat([EqualizedOdds_data,pd.DataFrame([EqualizedOdss_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualizedOdds_data.columns.name="Equalized Odds"
    display(EqualizedOdds_data)


In [42]:
#Train RandomForest as Baseline model on original Datasets
def baseline_CV(train_set, test_set):
    #Training
    baseline=RandomForestClassifier()
    baseline.fit(train_set.features,train_set.labels.ravel())
    #Testing
    pred=baseline.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Baseline",baseline_CV)

Accuracy  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline             0.895              0.94             0.905   

Accuracy  Cross Validate 4  Cross Validate 5  Average  
Baseline             0.895              0.86    0.899

Disparate Impact  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                  1.042549          1.457706          0.851852   

Disparate Impact  Cross Validate 4  Cross Validate 5   Average  
Baseline                  0.999406          1.039602  1.078223

Disparate Parity  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                   0.02272          0.177037             -0.08   

Disparate Parity  Cross Validate 4  Cross Validate 5   Average  
Baseline                   -0.0003          0.019256  0.027743

Equal Opportuniy  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                 -0.011574         -0.017857           0.00076   

Equal Opportuniy  Cross Validate 4  Cross Validate 5   Average  
Baseline                  0.098039          0.060823  0.026038

Equalized Odds  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline               -0.088795          0.021382         -0.015437   

Equalized Odds  Cross Validate 4  Cross Validate 5   Average  
Baseline               -0.053512         -0.012061 -0.029685

In [43]:
#Zhange et al. implementation
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
def AdversarialDebiase_CV(train_set,test_set):
    #Training
    tf.disable_eager_execution()
    tf.reset_default_graph()
    with tf.Session() as sess:
        debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
        debiased_model.fit(train_set)
    #Testing
        test_pred=debiased_model.predict(test_set)
    return test_pred

cross_validation("AdversarialDebiase",AdversarialDebiase_CV)

epoch 0; iter: 0; batch classifier loss: 0.709463; batch adversarial loss: 0.730790
epoch 1; iter: 0; batch classifier loss: 0.688165; batch adversarial loss: 0.690603
epoch 2; iter: 0; batch classifier loss: 0.668382; batch adversarial loss: 0.731945
epoch 3; iter: 0; batch classifier loss: 0.655802; batch adversarial loss: 0.713726
epoch 4; iter: 0; batch classifier loss: 0.655800; batch adversarial loss: 0.721309
epoch 5; iter: 0; batch classifier loss: 0.621499; batch adversarial loss: 0.715190
epoch 6; iter: 0; batch classifier loss: 0.623288; batch adversarial loss: 0.718986
epoch 7; iter: 0; batch classifier loss: 0.606301; batch adversarial loss: 0.705539
epoch 8; iter: 0; batch classifier loss: 0.618964; batch adversarial loss: 0.704291
epoch 9; iter: 0; batch classifier loss: 0.580367; batch adversarial loss: 0.701665
epoch 10; iter: 0; batch classifier loss: 0.569378; batch adversarial loss: 0.693609
epoch 11; iter: 0; batch classifier loss: 0.566493; batch adversarial loss:

Accuracy            Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                       0.895              0.94             0.905   
AdversarialDebiase             0.860              0.84             0.810   

Accuracy            Cross Validate 4  Cross Validate 5  Average  
Baseline                       0.895              0.86    0.899  
AdversarialDebiase             0.870              0.53    0.782

Disparate Impact    Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                    1.042549          1.457706          0.851852   
AdversarialDebiase          1.104330          2.437200          1.609756   

Disparate Impact    Cross Validate 4  Cross Validate 5   Average  
Baseline                    0.999406          1.039602  1.078223  
AdversarialDebiase          0.837109          0.044778  1.206634

Disparate Parity    Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                    0.022720          0.177037             -0.08   
AdversarialDebiase          0.050646          0.420313              0.25   

Disparate Parity    Cross Validate 4  Cross Validate 5   Average  
Baseline                   -0.000300          0.019256  0.027743  
AdversarialDebiase         -0.090682         -0.937695 -0.061484

Equal Opportuniy    Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                   -0.011574         -0.017857          0.000760   
AdversarialDebiase          0.055556          0.255952          0.303571   

Equal Opportuniy    Cross Validate 4  Cross Validate 5   Average  
Baseline                    0.098039          0.060823  0.026038  
AdversarialDebiase         -0.019608         -0.906977 -0.062301

Equalized Odds      Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                   -0.088795          0.021382         -0.015437   
AdversarialDebiase         -0.088795          0.337171          0.313036   

Equalized Odds      Cross Validate 4  Cross Validate 5   Average  
Baseline                   -0.053512         -0.012061 -0.029685  
AdversarialDebiase         -0.118729         -0.964912 -0.104446

In [44]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation
from aif360.algorithms.inprocessing import GridSearchReduction
def GridSearchReduction_EO_CV(train_set,test_set):
    #Training
    gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="gender")
    gsr_eo_model.fit(train_set)
    #Testing
    test_pred=gsr_eo_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(EO)",GridSearchReduction_EO_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy                 Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                            0.895              0.94             0.905   
AdversarialDebiase                  0.860              0.84             0.810   
GridSearchReduction(EO)             0.875              0.94             0.910   

Accuracy                 Cross Validate 4  Cross Validate 5  Average  
Baseline                            0.895              0.86    0.899  
AdversarialDebiase                  0.870              0.53    0.782  
GridSearchReduction(EO)             0.905              0.89    0.904

Disparate Impact         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         1.042549          1.457706          0.851852   
AdversarialDebiase               1.104330          2.437200          1.609756   
GridSearchReduction(EO)          1.005969          1.449848          0.836364   

Disparate Impact         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.999406          1.039602  1.078223  
AdversarialDebiase               0.837109          0.044778  1.206634  
GridSearchReduction(EO)          0.998252          0.994402  1.056967

Disparate Parity         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.022720          0.177037             -0.08   
AdversarialDebiase               0.050646          0.420313              0.25   
GridSearchReduction(EO)          0.003303          0.178242             -0.09   

Disparate Parity         Cross Validate 4  Cross Validate 5   Average  
Baseline                        -0.000300          0.019256  0.027743  
AdversarialDebiase              -0.090682         -0.937695 -0.061484  
GridSearchReduction(EO)         -0.000901         -0.002722  0.017584

Equal Opportuniy         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                        -0.011574         -0.017857          0.000760   
AdversarialDebiase               0.055556          0.255952          0.303571   
GridSearchReduction(EO)         -0.030093          0.000000         -0.017097   

Equal Opportuniy         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.098039          0.060823  0.026038  
AdversarialDebiase              -0.019608         -0.906977 -0.062301  
GridSearchReduction(EO)          0.098039         -0.020125  0.006145

Equalized Odds           Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                        -0.088795          0.021382         -0.015437   
AdversarialDebiase              -0.088795          0.337171          0.313036   
GridSearchReduction(EO)         -0.101903          0.005757         -0.015437   

Equalized Odds           Cross Validate 4  Cross Validate 5   Average  
Baseline                        -0.053512         -0.012061 -0.029685  
AdversarialDebiase              -0.118729         -0.964912 -0.104446  
GridSearchReduction(EO)         -0.053512          0.019737 -0.029072

In [45]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation

def GridSearchReduction_DP_CV(train_set,test_set):
    #Training
    gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="gender")
    gsr_dp_model.fit(train_set)
    #Testing
    test_pred=gsr_dp_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(DP)",GridSearchReduction_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy                 Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                            0.895             0.940             0.905   
AdversarialDebiase                  0.860             0.840             0.810   
GridSearchReduction(EO)             0.875             0.940             0.910   
GridSearchReduction(DP)             0.890             0.925             0.905   

Accuracy                 Cross Validate 4  Cross Validate 5  Average  
Baseline                            0.895             0.860    0.899  
AdversarialDebiase                  0.870             0.530    0.782  
GridSearchReduction(EO)             0.905             0.890    0.904  
GridSearchReduction(DP)             0.910             0.885    0.903

Disparate Impact         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         1.042549          1.457706          0.851852   
AdversarialDebiase               1.104330          2.437200          1.609756   
GridSearchReduction(EO)          1.005969          1.449848          0.836364   
GridSearchReduction(DP)          0.986009          1.430202          0.851852   

Disparate Impact         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.999406          1.039602  1.078223  
AdversarialDebiase               0.837109          0.044778  1.206634  
GridSearchReduction(EO)          0.998252          0.994402  1.056967  
GridSearchReduction(DP)          1.018625          0.945633  1.046464

Disparate Parity         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.022720          0.177037             -0.08   
AdversarialDebiase               0.050646          0.420313              0.25   
GridSearchReduction(EO)          0.003303          0.178242             -0.09   
GridSearchReduction(DP)         -0.007607          0.166399             -0.08   

Disparate Parity         Cross Validate 4  Cross Validate 5   Average  
Baseline                        -0.000300          0.019256  0.027743  
AdversarialDebiase              -0.090682         -0.937695 -0.061484  
GridSearchReduction(EO)         -0.000901         -0.002722  0.017584  
GridSearchReduction(DP)          0.009408         -0.028430  0.011954

Equal Opportuniy         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                        -0.011574         -0.017857          0.000760   
AdversarialDebiase               0.055556          0.255952          0.303571   
GridSearchReduction(EO)         -0.030093          0.000000         -0.017097   
GridSearchReduction(DP)         -0.030093          0.029762          0.000760   

Equal Opportuniy         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.098039          0.060823  0.026038  
AdversarialDebiase              -0.019608         -0.906977 -0.062301  
GridSearchReduction(EO)          0.098039         -0.020125  0.006145  
GridSearchReduction(DP)          0.098039         -0.016100  0.016474

Equalized Odds           Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                        -0.088795          0.021382         -0.015437   
AdversarialDebiase              -0.088795          0.337171          0.313036   
GridSearchReduction(EO)         -0.101903          0.005757         -0.015437   
GridSearchReduction(DP)         -0.130233         -0.036184         -0.015437   

Equalized Odds           Cross Validate 4  Cross Validate 5   Average  
Baseline                        -0.053512         -0.012061 -0.029685  
AdversarialDebiase              -0.118729         -0.964912 -0.104446  
GridSearchReduction(EO)         -0.053512          0.019737 -0.029072  
GridSearchReduction(DP)         -0.031773         -0.032895 -0.049304

In [46]:
#Kamiran and Calders, 2012 implementation
from aif360.algorithms.preprocessing import Reweighing
def Reweight_CV(train_set,test_set):
    #Training
    reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
    reweight.fit(train_set)
    reweight_set=reweight.transform(train_set.copy(deepcopy=True))
    reweight_test = reweight.transform(test_set.copy(deepcopy=True))

    reweight_model=RandomForestClassifier()
    reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)
    #Testing
    pred=reweight_model.predict(reweight_test.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Reweight",Reweight_CV)

Accuracy                 Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                            0.895             0.940             0.905   
AdversarialDebiase                  0.860             0.840             0.810   
GridSearchReduction(EO)             0.875             0.940             0.910   
GridSearchReduction(DP)             0.890             0.925             0.905   
Reweight                            0.890             0.935             0.910   

Accuracy                 Cross Validate 4  Cross Validate 5  Average  
Baseline                            0.895             0.860    0.899  
AdversarialDebiase                  0.870             0.530    0.782  
GridSearchReduction(EO)             0.905             0.890    0.904  
GridSearchReduction(DP)             0.910             0.885    0.903  
Reweight                            0.900             0.880    0.903

Disparate Impact         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         1.042549          1.457706          0.851852   
AdversarialDebiase               1.104330          2.437200          1.609756   
GridSearchReduction(EO)          1.005969          1.449848          0.836364   
GridSearchReduction(DP)          0.986009          1.430202          0.851852   
Reweight                         1.022528          1.422999          0.942308   

Disparate Impact         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.999406          1.039602  1.078223  
AdversarialDebiase               0.837109          0.044778  1.206634  
GridSearchReduction(EO)          0.998252          0.994402  1.056967  
GridSearchReduction(DP)          1.018625          0.945633  1.046464  
Reweight                         0.979417          1.078022  1.089055

Disparate Parity         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.022720          0.177037             -0.08   
AdversarialDebiase               0.050646          0.420313              0.25   
GridSearchReduction(EO)          0.003303          0.178242             -0.09   
GridSearchReduction(DP)         -0.007607          0.166399             -0.08   
Reweight                         0.011811          0.167603             -0.03   

Disparate Parity         Cross Validate 4  Cross Validate 5   Average  
Baseline                        -0.000300          0.019256  0.027743  
AdversarialDebiase              -0.090682         -0.937695 -0.061484  
GridSearchReduction(EO)         -0.000901         -0.002722  0.017584  
GridSearchReduction(DP)          0.009408         -0.028430  0.011954  
Reweight                        -0.010610          0.035790  0.034919

Equal Opportuniy         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                        -0.011574         -0.017857          0.000760   
AdversarialDebiase               0.055556          0.255952          0.303571   
GridSearchReduction(EO)         -0.030093          0.000000         -0.017097   
GridSearchReduction(DP)         -0.030093          0.029762          0.000760   
Reweight                        -0.009259         -0.017857          0.100304   

Equal Opportuniy         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.098039          0.060823  0.026038  
AdversarialDebiase              -0.019608         -0.906977 -0.062301  
GridSearchReduction(EO)          0.098039         -0.020125  0.006145  
GridSearchReduction(DP)          0.098039         -0.016100  0.016474  
Reweight                         0.078431          0.060823  0.042488

Equalized Odds           Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                        -0.088795          0.021382         -0.015437   
AdversarialDebiase              -0.088795          0.337171          0.313036   
GridSearchReduction(EO)         -0.101903          0.005757         -0.015437   
GridSearchReduction(DP)         -0.130233         -0.036184         -0.015437   
Reweight                        -0.112051          0.005757         -0.015437   

Equalized Odds           Cross Validate 4  Cross Validate 5   Average  
Baseline                        -0.053512         -0.012061 -0.029685  
AdversarialDebiase              -0.118729         -0.964912 -0.104446  
GridSearchReduction(EO)         -0.053512          0.019737 -0.029072  
GridSearchReduction(DP)         -0.031773         -0.032895 -0.049304  
Reweight                        -0.053512          0.019737 -0.031101

In [47]:
#Feldman et al., 2015 implementation
from aif360.algorithms.preprocessing import DisparateImpactRemover

def DisparateImpactRemover_05_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.5)",DisparateImpactRemover_05_CV)

Accuracy                     Cross Validate 1  Cross Validate 2  \
Baseline                                0.895             0.940   
AdversarialDebiase                      0.860             0.840   
GridSearchReduction(EO)                 0.875             0.940   
GridSearchReduction(DP)                 0.890             0.925   
Reweight                                0.890             0.935   
DisparateImpactRemover(0.5)             0.880             0.930   

Accuracy                     Cross Validate 3  Cross Validate 4  \
Baseline                                0.905             0.895   
AdversarialDebiase                      0.810             0.870   
GridSearchReduction(EO)                 0.910             0.905   
GridSearchReduction(DP)                 0.905             0.910   
Reweight                                0.910             0.900   
DisparateImpactRemover(0.5)             0.910             0.910   

Accuracy                     Cross Validate 5  Average  
Baseline                                0.860    0.899  
AdversarialDebiase                      0.530    0.782  
GridSearchReduction(EO)                 0.890    0.904  
GridSearchReduction(DP)                 0.885    0.903  
Reweight                                0.880    0.903  
DisparateImpactRemover(0.5)             0.900    0.906

Disparate Impact             Cross Validate 1  Cross Validate 2  \
Baseline                             1.042549          1.457706   
AdversarialDebiase                   1.104330          2.437200   
GridSearchReduction(EO)              1.005969          1.449848   
GridSearchReduction(DP)              0.986009          1.430202   
Reweight                             1.022528          1.422999   
DisparateImpactRemover(0.5)          1.021786          1.532460   

Disparate Impact             Cross Validate 3  Cross Validate 4  \
Baseline                             0.851852          0.999406   
AdversarialDebiase                   1.609756          0.837109   
GridSearchReduction(EO)              0.836364          0.998252   
GridSearchReduction(DP)              0.851852          1.018625   
Reweight                             0.942308          0.979417   
DisparateImpactRemover(0.5)          0.872727          1.018625   

Disparate Impact             Cross Validate 5   Average  
Baseline                             1.039602  1.078223  
AdversarialDebiase                   0.044778  1.206634  
GridSearchReduction(EO)              0.994402  1.056967  
GridSearchReduction(DP)              0.945633  1.046464  
Reweight                             1.078022  1.089055  
DisparateImpactRemover(0.5)          1.033398  1.095799

Disparate Parity             Cross Validate 1  Cross Validate 2  \
Baseline                             0.022720          0.177037   
AdversarialDebiase                   0.050646          0.420313   
GridSearchReduction(EO)              0.003303          0.178242   
GridSearchReduction(DP)             -0.007607          0.166399   
Reweight                             0.011811          0.167603   
DisparateImpactRemover(0.5)          0.011210          0.195905   

Disparate Parity             Cross Validate 3  Cross Validate 4  \
Baseline                                -0.08         -0.000300   
AdversarialDebiase                       0.25         -0.090682   
GridSearchReduction(EO)                 -0.09         -0.000901   
GridSearchReduction(DP)                 -0.08          0.009408   
Reweight                                -0.03         -0.010610   
DisparateImpactRemover(0.5)             -0.07          0.009408   

Disparate Parity             Cross Validate 5   Average  
Baseline                             0.019256  0.027743  
AdversarialDebiase                  -0.937695 -0.061484  
GridSearchReduction(EO)             -0.002722  0.017584  
GridSearchReduction(DP)             -0.028430  0.011954  
Reweight                             0.035790  0.034919  
DisparateImpactRemover(0.5)          0.015627  0.032430

Equal Opportuniy             Cross Validate 1  Cross Validate 2  \
Baseline                            -0.011574         -0.017857   
AdversarialDebiase                   0.055556          0.255952   
GridSearchReduction(EO)             -0.030093          0.000000   
GridSearchReduction(DP)             -0.030093          0.029762   
Reweight                            -0.009259         -0.017857   
DisparateImpactRemover(0.5)         -0.006944          0.029762   

Equal Opportuniy             Cross Validate 3  Cross Validate 4  \
Baseline                             0.000760          0.098039   
AdversarialDebiase                   0.303571         -0.019608   
GridSearchReduction(EO)             -0.017097          0.098039   
GridSearchReduction(DP)              0.000760          0.098039   
Reweight                             0.100304          0.078431   
DisparateImpactRemover(0.5)          0.043313          0.098039   

Equal Opportuniy             Cross Validate 5   Average  
Baseline                             0.060823  0.026038  
AdversarialDebiase                  -0.906977 -0.062301  
GridSearchReduction(EO)             -0.020125  0.006145  
GridSearchReduction(DP)             -0.016100  0.016474  
Reweight                             0.060823  0.042488  
DisparateImpactRemover(0.5)          0.022361  0.037306

Equalized Odds               Cross Validate 1  Cross Validate 2  \
Baseline                            -0.088795          0.021382   
AdversarialDebiase                  -0.088795          0.337171   
GridSearchReduction(EO)             -0.101903          0.005757   
GridSearchReduction(DP)             -0.130233         -0.036184   
Reweight                            -0.112051          0.005757   
DisparateImpactRemover(0.5)         -0.112051          0.021382   

Equalized Odds               Cross Validate 3  Cross Validate 4  \
Baseline                            -0.015437         -0.053512   
AdversarialDebiase                   0.313036         -0.118729   
GridSearchReduction(EO)             -0.015437         -0.053512   
GridSearchReduction(DP)             -0.015437         -0.031773   
Reweight                            -0.015437         -0.053512   
DisparateImpactRemover(0.5)         -0.038165         -0.031773   

Equalized Odds               Cross Validate 5   Average  
Baseline                            -0.012061 -0.029685  
AdversarialDebiase                  -0.964912 -0.104446  
GridSearchReduction(EO)              0.019737 -0.029072  
GridSearchReduction(DP)             -0.032895 -0.049304  
Reweight                             0.019737 -0.031101  
DisparateImpactRemover(0.5)          0.016447 -0.028832

In [48]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_07_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.7)",DisparateImpactRemover_07_CV)

Accuracy                     Cross Validate 1  Cross Validate 2  \
Baseline                                0.895             0.940   
AdversarialDebiase                      0.860             0.840   
GridSearchReduction(EO)                 0.875             0.940   
GridSearchReduction(DP)                 0.890             0.925   
Reweight                                0.890             0.935   
DisparateImpactRemover(0.5)             0.880             0.930   
DisparateImpactRemover(0.7)             0.885             0.915   

Accuracy                     Cross Validate 3  Cross Validate 4  \
Baseline                                0.905             0.895   
AdversarialDebiase                      0.810             0.870   
GridSearchReduction(EO)                 0.910             0.905   
GridSearchReduction(DP)                 0.905             0.910   
Reweight                                0.910             0.900   
DisparateImpactRemover(0.5)             0.910             0.910   
DisparateImpactRemover(0.7)             0.890             0.900   

Accuracy                     Cross Validate 5  Average  
Baseline                                0.860    0.899  
AdversarialDebiase                      0.530    0.782  
GridSearchReduction(EO)                 0.890    0.904  
GridSearchReduction(DP)                 0.885    0.903  
Reweight                                0.880    0.903  
DisparateImpactRemover(0.5)             0.900    0.906  
DisparateImpactRemover(0.7)             0.885    0.895

Disparate Impact             Cross Validate 1  Cross Validate 2  \
Baseline                             1.042549          1.457706   
AdversarialDebiase                   1.104330          2.437200   
GridSearchReduction(EO)              1.005969          1.449848   
GridSearchReduction(DP)              0.986009          1.430202   
Reweight                             1.022528          1.422999   
DisparateImpactRemover(0.5)          1.021786          1.532460   
DisparateImpactRemover(0.7)          1.082676          1.315603   

Disparate Impact             Cross Validate 3  Cross Validate 4  \
Baseline                             0.851852          0.999406   
AdversarialDebiase                   1.609756          0.837109   
GridSearchReduction(EO)              0.836364          0.998252   
GridSearchReduction(DP)              0.851852          1.018625   
Reweight                             0.942308          0.979417   
DisparateImpactRemover(0.5)          0.872727          1.018625   
DisparateImpactRemover(0.7)          0.839286          1.109169   

Disparate Impact             Cross Validate 5   Average  
Baseline                             1.039602  1.078223  
AdversarialDebiase                   0.044778  1.206634  
GridSearchReduction(EO)              0.994402  1.056967  
GridSearchReduction(DP)              0.945633  1.046464  
Reweight                             1.078022  1.089055  
DisparateImpactRemover(0.5)          1.033398  1.095799  
DisparateImpactRemover(0.7)          1.054066  1.080160

Disparate Parity             Cross Validate 1  Cross Validate 2  \
Baseline                             0.022720          0.177037   
AdversarialDebiase                   0.050646          0.420313   
GridSearchReduction(EO)              0.003303          0.178242   
GridSearchReduction(DP)             -0.007607          0.166399   
Reweight                             0.011811          0.167603   
DisparateImpactRemover(0.5)          0.011210          0.195905   
DisparateImpactRemover(0.7)          0.040937          0.125050   

Disparate Parity             Cross Validate 3  Cross Validate 4  \
Baseline                                -0.08         -0.000300   
AdversarialDebiase                       0.25         -0.090682   
GridSearchReduction(EO)                 -0.09         -0.000901   
GridSearchReduction(DP)                 -0.08          0.009408   
Reweight                                -0.03         -0.010610   
DisparateImpactRemover(0.5)             -0.07          0.009408   
DisparateImpactRemover(0.7)             -0.09          0.050646   

Disparate Parity             Cross Validate 5   Average  
Baseline                             0.019256  0.027743  
AdversarialDebiase                  -0.937695 -0.061484  
GridSearchReduction(EO)             -0.002722  0.017584  
GridSearchReduction(DP)             -0.028430  0.011954  
Reweight                             0.035790  0.034919  
DisparateImpactRemover(0.5)          0.015627  0.032430  
DisparateImpactRemover(0.7)          0.024801  0.030287

Equal Opportuniy             Cross Validate 1  Cross Validate 2  \
Baseline                            -0.011574         -0.017857   
AdversarialDebiase                   0.055556          0.255952   
GridSearchReduction(EO)             -0.030093          0.000000   
GridSearchReduction(DP)             -0.030093          0.029762   
Reweight                            -0.009259         -0.017857   
DisparateImpactRemover(0.5)         -0.006944          0.029762   
DisparateImpactRemover(0.7)         -0.006944         -0.047619   

Equal Opportuniy             Cross Validate 3  Cross Validate 4  \
Baseline                             0.000760          0.098039   
AdversarialDebiase                   0.303571         -0.019608   
GridSearchReduction(EO)             -0.017097          0.098039   
GridSearchReduction(DP)              0.000760          0.098039   
Reweight                             0.100304          0.078431   
DisparateImpactRemover(0.5)          0.043313          0.098039   
DisparateImpactRemover(0.7)          0.000760          0.156863   

Equal Opportuniy             Cross Validate 5   Average  
Baseline                             0.060823  0.026038  
AdversarialDebiase                  -0.906977 -0.062301  
GridSearchReduction(EO)             -0.020125  0.006145  
GridSearchReduction(DP)             -0.016100  0.016474  
Reweight                             0.060823  0.042488  
DisparateImpactRemover(0.5)          0.022361  0.037306  
DisparateImpactRemover(0.7)          0.060823  0.032776

Equalized Odds               Cross Validate 1  Cross Validate 2  \
Baseline                            -0.088795          0.021382   
AdversarialDebiase                  -0.088795          0.337171   
GridSearchReduction(EO)             -0.101903          0.005757   
GridSearchReduction(DP)             -0.130233         -0.036184   
Reweight                            -0.112051          0.005757   
DisparateImpactRemover(0.5)         -0.112051          0.021382   
DisparateImpactRemover(0.7)         -0.052431         -0.036184   

Equalized Odds               Cross Validate 3  Cross Validate 4  \
Baseline                            -0.015437         -0.053512   
AdversarialDebiase                   0.313036         -0.118729   
GridSearchReduction(EO)             -0.015437         -0.053512   
GridSearchReduction(DP)             -0.015437         -0.031773   
Reweight                            -0.015437         -0.053512   
DisparateImpactRemover(0.5)         -0.038165         -0.031773   
DisparateImpactRemover(0.7)         -0.042024         -0.010033   

Equalized Odds               Cross Validate 5   Average  
Baseline                            -0.012061 -0.029685  
AdversarialDebiase                  -0.964912 -0.104446  
GridSearchReduction(EO)              0.019737 -0.029072  
GridSearchReduction(DP)             -0.032895 -0.049304  
Reweight                             0.019737 -0.031101  
DisparateImpactRemover(0.5)          0.016447 -0.028832  
DisparateImpactRemover(0.7)         -0.001096 -0.028354

In [49]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_10_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=1.0,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(1.0)",DisparateImpactRemover_10_CV)

Accuracy                     Cross Validate 1  Cross Validate 2  \
Baseline                                0.895             0.940   
AdversarialDebiase                      0.860             0.840   
GridSearchReduction(EO)                 0.875             0.940   
GridSearchReduction(DP)                 0.890             0.925   
Reweight                                0.890             0.935   
DisparateImpactRemover(0.5)             0.880             0.930   
DisparateImpactRemover(0.7)             0.885             0.915   
DisparateImpactRemover(1.0)             0.885             0.895   

Accuracy                     Cross Validate 3  Cross Validate 4  \
Baseline                                0.905             0.895   
AdversarialDebiase                      0.810             0.870   
GridSearchReduction(EO)                 0.910             0.905   
GridSearchReduction(DP)                 0.905             0.910   
Reweight                                0.910             0.900   
DisparateImpactRemover(0.5)             0.910             0.910   
DisparateImpactRemover(0.7)             0.890             0.900   
DisparateImpactRemover(1.0)             0.900             0.900   

Accuracy                     Cross Validate 5  Average  
Baseline                                0.860    0.899  
AdversarialDebiase                      0.530    0.782  
GridSearchReduction(EO)                 0.890    0.904  
GridSearchReduction(DP)                 0.885    0.903  
Reweight                                0.880    0.903  
DisparateImpactRemover(0.5)             0.900    0.906  
DisparateImpactRemover(0.7)             0.885    0.895  
DisparateImpactRemover(1.0)             0.890    0.894

Disparate Impact             Cross Validate 1  Cross Validate 2  \
Baseline                             1.042549          1.457706   
AdversarialDebiase                   1.104330          2.437200   
GridSearchReduction(EO)              1.005969          1.449848   
GridSearchReduction(DP)              0.986009          1.430202   
Reweight                             1.022528          1.422999   
DisparateImpactRemover(0.5)          1.021786          1.532460   
DisparateImpactRemover(0.7)          1.082676          1.315603   
DisparateImpactRemover(1.0)          1.082676          1.375195   

Disparate Impact             Cross Validate 3  Cross Validate 4  \
Baseline                             0.851852          0.999406   
AdversarialDebiase                   1.609756          0.837109   
GridSearchReduction(EO)              0.836364          0.998252   
GridSearchReduction(DP)              0.851852          1.018625   
Reweight                             0.942308          0.979417   
DisparateImpactRemover(0.5)          0.872727          1.018625   
DisparateImpactRemover(0.7)          0.839286          1.109169   
DisparateImpactRemover(1.0)          0.903846          1.059466   

Disparate Impact             Cross Validate 5   Average  
Baseline                             1.039602  1.078223  
AdversarialDebiase                   0.044778  1.206634  
GridSearchReduction(EO)              0.994402  1.056967  
GridSearchReduction(DP)              0.945633  1.046464  
Reweight                             1.078022  1.089055  
DisparateImpactRemover(0.5)          1.033398  1.095799  
DisparateImpactRemover(0.7)          1.054066  1.080160  
DisparateImpactRemover(1.0)          1.073031  1.098843

Disparate Parity             Cross Validate 1  Cross Validate 2  \
Baseline                             0.022720          0.177037   
AdversarialDebiase                   0.050646          0.420313   
GridSearchReduction(EO)              0.003303          0.178242   
GridSearchReduction(DP)             -0.007607          0.166399   
Reweight                             0.011811          0.167603   
DisparateImpactRemover(0.5)          0.011210          0.195905   
DisparateImpactRemover(0.7)          0.040937          0.125050   
DisparateImpactRemover(1.0)          0.040937          0.145122   

Disparate Parity             Cross Validate 3  Cross Validate 4  \
Baseline                                -0.08         -0.000300   
AdversarialDebiase                       0.25         -0.090682   
GridSearchReduction(EO)                 -0.09         -0.000901   
GridSearchReduction(DP)                 -0.08          0.009408   
Reweight                                -0.03         -0.010610   
DisparateImpactRemover(0.5)             -0.07          0.009408   
DisparateImpactRemover(0.7)             -0.09          0.050646   
DisparateImpactRemover(1.0)             -0.05          0.029426   

Disparate Parity             Cross Validate 5   Average  
Baseline                             0.019256  0.027743  
AdversarialDebiase                  -0.937695 -0.061484  
GridSearchReduction(EO)             -0.002722  0.017584  
GridSearchReduction(DP)             -0.028430  0.011954  
Reweight                             0.035790  0.034919  
DisparateImpactRemover(0.5)          0.015627  0.032430  
DisparateImpactRemover(0.7)          0.024801  0.030287  
DisparateImpactRemover(1.0)          0.032161  0.039529

Equal Opportuniy             Cross Validate 1  Cross Validate 2  \
Baseline                            -0.011574         -0.017857   
AdversarialDebiase                   0.055556          0.255952   
GridSearchReduction(EO)             -0.030093          0.000000   
GridSearchReduction(DP)             -0.030093          0.029762   
Reweight                            -0.009259         -0.017857   
DisparateImpactRemover(0.5)         -0.006944          0.029762   
DisparateImpactRemover(0.7)         -0.006944         -0.047619   
DisparateImpactRemover(1.0)         -0.006944          0.000000   

Equal Opportuniy             Cross Validate 3  Cross Validate 4  \
Baseline                             0.000760          0.098039   
AdversarialDebiase                   0.303571         -0.019608   
GridSearchReduction(EO)             -0.017097          0.098039   
GridSearchReduction(DP)              0.000760          0.098039   
Reweight                             0.100304          0.078431   
DisparateImpactRemover(0.5)          0.043313          0.098039   
DisparateImpactRemover(0.7)          0.000760          0.156863   
DisparateImpactRemover(1.0)          0.057751          0.117647   

Equal Opportuniy             Cross Validate 5   Average  
Baseline                             0.060823  0.026038  
AdversarialDebiase                  -0.906977 -0.062301  
GridSearchReduction(EO)             -0.020125  0.006145  
GridSearchReduction(DP)             -0.016100  0.016474  
Reweight                             0.060823  0.042488  
DisparateImpactRemover(0.5)          0.022361  0.037306  
DisparateImpactRemover(0.7)          0.060823  0.032776  
DisparateImpactRemover(1.0)          0.037567  0.041204

Equalized Odds               Cross Validate 1  Cross Validate 2  \
Baseline                            -0.088795          0.021382   
AdversarialDebiase                  -0.088795          0.337171   
GridSearchReduction(EO)             -0.101903          0.005757   
GridSearchReduction(DP)             -0.130233         -0.036184   
Reweight                            -0.112051          0.005757   
DisparateImpactRemover(0.5)         -0.112051          0.021382   
DisparateImpactRemover(0.7)         -0.052431         -0.036184   
DisparateImpactRemover(1.0)         -0.052431         -0.025493   

Equalized Odds               Cross Validate 3  Cross Validate 4  \
Baseline                            -0.015437         -0.053512   
AdversarialDebiase                   0.313036         -0.118729   
GridSearchReduction(EO)             -0.015437         -0.053512   
GridSearchReduction(DP)             -0.015437         -0.031773   
Reweight                            -0.015437         -0.053512   
DisparateImpactRemover(0.5)         -0.038165         -0.031773   
DisparateImpactRemover(0.7)         -0.042024         -0.010033   
DisparateImpactRemover(1.0)         -0.015437         -0.012542   

Equalized Odds               Cross Validate 5   Average  
Baseline                            -0.012061 -0.029685  
AdversarialDebiase                  -0.964912 -0.104446  
GridSearchReduction(EO)              0.019737 -0.029072  
GridSearchReduction(DP)             -0.032895 -0.049304  
Reweight                             0.019737 -0.031101  
DisparateImpactRemover(0.5)          0.016447 -0.028832  
DisparateImpactRemover(0.7)         -0.001096 -0.028354  
DisparateImpactRemover(1.0)          0.033991 -0.014383

In [50]:
#Pleiss et al., 2017 implementation
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

def CalibratedEqOddsPostprocessing_CV(train_set,test_set):
    #Training
    ceop_model=RandomForestClassifier()
    ceop_model.fit(train_set.features,train_set.labels.ravel())

    train_eop_set=train_set.copy(deepcopy=True)
    train_eop_set.scores=ceop_model.predict(train_set.features)

    ceop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
    ceop.fit(train_set,train_eop_set)
    #Testing
    pred=ceop_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=ceop.predict(test_pred)
    return test_pred

cross_validation("CalibratedEqOddsPostprocessing",CalibratedEqOddsPostprocessing_CV)

Accuracy                        Cross Validate 1  Cross Validate 2  \
Baseline                                   0.895             0.940   
AdversarialDebiase                         0.860             0.840   
GridSearchReduction(EO)                    0.875             0.940   
GridSearchReduction(DP)                    0.890             0.925   
Reweight                                   0.890             0.935   
DisparateImpactRemover(0.5)                0.880             0.930   
DisparateImpactRemover(0.7)                0.885             0.915   
DisparateImpactRemover(1.0)                0.885             0.895   
CalibratedEqOddsPostprocessing             0.885             0.935   

Accuracy                        Cross Validate 3  Cross Validate 4  \
Baseline                                   0.905             0.895   
AdversarialDebiase                         0.810             0.870   
GridSearchReduction(EO)                    0.910             0.905   
GridSearchReduction(DP)                    0.905             0.910   
Reweight                                   0.910             0.900   
DisparateImpactRemover(0.5)                0.910             0.910   
DisparateImpactRemover(0.7)                0.890             0.900   
DisparateImpactRemover(1.0)                0.900             0.900   
CalibratedEqOddsPostprocessing             0.905             0.910   

Accuracy                        Cross Validate 5  Average  
Baseline                                   0.860    0.899  
AdversarialDebiase                         0.530    0.782  
GridSearchReduction(EO)                    0.890    0.904  
GridSearchReduction(DP)                    0.885    0.903  
Reweight                                   0.880    0.903  
DisparateImpactRemover(0.5)                0.900    0.906  
DisparateImpactRemover(0.7)                0.885    0.895  
DisparateImpactRemover(1.0)                0.890    0.894  
CalibratedEqOddsPostprocessing             0.875    0.902

Disparate Impact                Cross Validate 1  Cross Validate 2  \
Baseline                                1.042549          1.457706   
AdversarialDebiase                      1.104330          2.437200   
GridSearchReduction(EO)                 1.005969          1.449848   
GridSearchReduction(DP)                 0.986009          1.430202   
Reweight                                1.022528          1.422999   
DisparateImpactRemover(0.5)             1.021786          1.532460   
DisparateImpactRemover(0.7)             1.082676          1.315603   
DisparateImpactRemover(1.0)             1.082676          1.375195   
CalibratedEqOddsPostprocessing          1.081891          1.485210   

Disparate Impact                Cross Validate 3  Cross Validate 4  \
Baseline                                0.851852          0.999406   
AdversarialDebiase                      1.609756          0.837109   
GridSearchReduction(EO)                 0.836364          0.998252   
GridSearchReduction(DP)                 0.851852          1.018625   
Reweight                                0.942308          0.979417   
DisparateImpactRemover(0.5)             0.872727          1.018625   
DisparateImpactRemover(0.7)             0.839286          1.109169   
DisparateImpactRemover(1.0)             0.903846          1.059466   
CalibratedEqOddsPostprocessing          0.888889          1.059466   

Disparate Impact                Cross Validate 5   Average  
Baseline                                1.039602  1.078223  
AdversarialDebiase                      0.044778  1.206634  
GridSearchReduction(EO)                 0.994402  1.056967  
GridSearchReduction(DP)                 0.945633  1.046464  
Reweight                                1.078022  1.089055  
DisparateImpactRemover(0.5)             1.033398  1.095799  
DisparateImpactRemover(0.7)             1.054066  1.080160  
DisparateImpactRemover(1.0)             1.073031  1.098843  
CalibratedEqOddsPostprocessing          1.101978  1.123487

Disparate Parity                Cross Validate 1  Cross Validate 2  \
Baseline                                0.022720          0.177037   
AdversarialDebiase                      0.050646          0.420313   
GridSearchReduction(EO)                 0.003303          0.178242   
GridSearchReduction(DP)                -0.007607          0.166399   
Reweight                                0.011811          0.167603   
DisparateImpactRemover(0.5)             0.011210          0.195905   
DisparateImpactRemover(0.7)             0.040937          0.125050   
DisparateImpactRemover(1.0)             0.040937          0.145122   
CalibratedEqOddsPostprocessing          0.042138          0.187676   

Disparate Parity                Cross Validate 3  Cross Validate 4  \
Baseline                                   -0.08         -0.000300   
AdversarialDebiase                          0.25         -0.090682   
GridSearchReduction(EO)                    -0.09         -0.000901   
GridSearchReduction(DP)                    -0.08          0.009408   
Reweight                                   -0.03         -0.010610   
DisparateImpactRemover(0.5)                -0.07          0.009408   
DisparateImpactRemover(0.7)                -0.09          0.050646   
DisparateImpactRemover(1.0)                -0.05          0.029426   
CalibratedEqOddsPostprocessing             -0.06          0.029426   

Disparate Parity                Cross Validate 5   Average  
Baseline                                0.019256  0.027743  
AdversarialDebiase                     -0.937695 -0.061484  
GridSearchReduction(EO)                -0.002722  0.017584  
GridSearchReduction(DP)                -0.028430  0.011954  
Reweight                                0.035790  0.034919  
DisparateImpactRemover(0.5)             0.015627  0.032430  
DisparateImpactRemover(0.7)             0.024801  0.030287  
DisparateImpactRemover(1.0)             0.032161  0.039529  
CalibratedEqOddsPostprocessing          0.046779  0.049204

Equal Opportuniy                Cross Validate 1  Cross Validate 2  \
Baseline                               -0.011574         -0.017857   
AdversarialDebiase                      0.055556          0.255952   
GridSearchReduction(EO)                -0.030093          0.000000   
GridSearchReduction(DP)                -0.030093          0.029762   
Reweight                               -0.009259         -0.017857   
DisparateImpactRemover(0.5)            -0.006944          0.029762   
DisparateImpactRemover(0.7)            -0.006944         -0.047619   
DisparateImpactRemover(1.0)            -0.006944          0.000000   
CalibratedEqOddsPostprocessing          0.030093          0.023810   

Equal Opportuniy                Cross Validate 3  Cross Validate 4  \
Baseline                                0.000760          0.098039   
AdversarialDebiase                      0.303571         -0.019608   
GridSearchReduction(EO)                -0.017097          0.098039   
GridSearchReduction(DP)                 0.000760          0.098039   
Reweight                                0.100304          0.078431   
DisparateImpactRemover(0.5)             0.043313          0.098039   
DisparateImpactRemover(0.7)             0.000760          0.156863   
DisparateImpactRemover(1.0)             0.057751          0.117647   
CalibratedEqOddsPostprocessing          0.022036          0.098039   

Equal Opportuniy                Cross Validate 5   Average  
Baseline                                0.060823  0.026038  
AdversarialDebiase                     -0.906977 -0.062301  
GridSearchReduction(EO)                -0.020125  0.006145  
GridSearchReduction(DP)                -0.016100  0.016474  
Reweight                                0.060823  0.042488  
DisparateImpactRemover(0.5)             0.022361  0.037306  
DisparateImpactRemover(0.7)             0.060823  0.032776  
DisparateImpactRemover(1.0)             0.037567  0.041204  
CalibratedEqOddsPostprocessing          0.060823  0.046960

Equalized Odds                  Cross Validate 1  Cross Validate 2  \
Baseline                               -0.088795          0.021382   
AdversarialDebiase                     -0.088795          0.337171   
GridSearchReduction(EO)                -0.101903          0.005757   
GridSearchReduction(DP)                -0.130233         -0.036184   
Reweight                               -0.112051          0.005757   
DisparateImpactRemover(0.5)            -0.112051          0.021382   
DisparateImpactRemover(0.7)            -0.052431         -0.036184   
DisparateImpactRemover(1.0)            -0.052431         -0.025493   
CalibratedEqOddsPostprocessing         -0.088795          0.005757   

Equalized Odds                  Cross Validate 3  Cross Validate 4  \
Baseline                               -0.015437         -0.053512   
AdversarialDebiase                      0.313036         -0.118729   
GridSearchReduction(EO)                -0.015437         -0.053512   
GridSearchReduction(DP)                -0.015437         -0.031773   
Reweight                               -0.015437         -0.053512   
DisparateImpactRemover(0.5)            -0.038165         -0.031773   
DisparateImpactRemover(0.7)            -0.042024         -0.010033   
DisparateImpactRemover(1.0)            -0.015437         -0.012542   
CalibratedEqOddsPostprocessing          0.003431          0.009197   

Equalized Odds                  Cross Validate 5   Average  
Baseline                               -0.012061 -0.029685  
AdversarialDebiase                     -0.964912 -0.104446  
GridSearchReduction(EO)                 0.019737 -0.029072  
GridSearchReduction(DP)                -0.032895 -0.049304  
Reweight                                0.019737 -0.031101  
DisparateImpactRemover(0.5)             0.016447 -0.028832  
DisparateImpactRemover(0.7)            -0.001096 -0.028354  
DisparateImpactRemover(1.0)             0.033991 -0.014383  
CalibratedEqOddsPostprocessing          0.040570 -0.005968

In [51]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_EO_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Equal opportunity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(EO)",RejectOptionClassification_EO_CV)

Accuracy                        Cross Validate 1  Cross Validate 2  \
Baseline                                   0.895             0.940   
AdversarialDebiase                         0.860             0.840   
GridSearchReduction(EO)                    0.875             0.940   
GridSearchReduction(DP)                    0.890             0.925   
Reweight                                   0.890             0.935   
DisparateImpactRemover(0.5)                0.880             0.930   
DisparateImpactRemover(0.7)                0.885             0.915   
DisparateImpactRemover(1.0)                0.885             0.895   
CalibratedEqOddsPostprocessing             0.885             0.935   
RejectOptionClassification(EO)             0.885             0.940   

Accuracy                        Cross Validate 3  Cross Validate 4  \
Baseline                                   0.905             0.895   
AdversarialDebiase                         0.810             0.870   
GridSearchReduction(EO)                    0.910             0.905   
GridSearchReduction(DP)                    0.905             0.910   
Reweight                                   0.910             0.900   
DisparateImpactRemover(0.5)                0.910             0.910   
DisparateImpactRemover(0.7)                0.890             0.900   
DisparateImpactRemover(1.0)                0.900             0.900   
CalibratedEqOddsPostprocessing             0.905             0.910   
RejectOptionClassification(EO)             0.905             0.890   

Accuracy                        Cross Validate 5  Average  
Baseline                                   0.860    0.899  
AdversarialDebiase                         0.530    0.782  
GridSearchReduction(EO)                    0.890    0.904  
GridSearchReduction(DP)                    0.885    0.903  
Reweight                                   0.880    0.903  
DisparateImpactRemover(0.5)                0.900    0.906  
DisparateImpactRemover(0.7)                0.885    0.895  
DisparateImpactRemover(1.0)                0.890    0.894  
CalibratedEqOddsPostprocessing             0.875    0.902  
RejectOptionClassification(EO)             0.895    0.903

Disparate Impact                Cross Validate 1  Cross Validate 2  \
Baseline                                1.042549          1.457706   
AdversarialDebiase                      1.104330          2.437200   
GridSearchReduction(EO)                 1.005969          1.449848   
GridSearchReduction(DP)                 0.986009          1.430202   
Reweight                                1.022528          1.422999   
DisparateImpactRemover(0.5)             1.021786          1.532460   
DisparateImpactRemover(0.7)             1.082676          1.315603   
DisparateImpactRemover(1.0)             1.082676          1.375195   
CalibratedEqOddsPostprocessing          1.081891          1.485210   
RejectOptionClassification(EO)          1.004971          1.449848   

Disparate Impact                Cross Validate 3  Cross Validate 4  \
Baseline                                0.851852          0.999406   
AdversarialDebiase                      1.609756          0.837109   
GridSearchReduction(EO)                 0.836364          0.998252   
GridSearchReduction(DP)                 0.851852          1.018625   
Reweight                                0.942308          0.979417   
DisparateImpactRemover(0.5)             0.872727          1.018625   
DisparateImpactRemover(0.7)             0.839286          1.109169   
DisparateImpactRemover(1.0)             0.903846          1.059466   
CalibratedEqOddsPostprocessing          0.888889          1.059466   
RejectOptionClassification(EO)          0.886792          0.978679   

Disparate Impact                Cross Validate 5   Average  
Baseline                                1.039602  1.078223  
AdversarialDebiase                      0.044778  1.206634  
GridSearchReduction(EO)                 0.994402  1.056967  
GridSearchReduction(DP)                 0.945633  1.046464  
Reweight                                1.078022  1.089055  
DisparateImpactRemover(0.5)             1.033398  1.095799  
DisparateImpactRemover(0.7)             1.054066  1.080160  
DisparateImpactRemover(1.0)             1.073031  1.098843  
CalibratedEqOddsPostprocessing          1.101978  1.123487  
RejectOptionClassification(EO)          1.148912  1.093840

Disparate Parity                Cross Validate 1  Cross Validate 2  \
Baseline                                0.022720          0.177037   
AdversarialDebiase                      0.050646          0.420313   
GridSearchReduction(EO)                 0.003303          0.178242   
GridSearchReduction(DP)                -0.007607          0.166399   
Reweight                                0.011811          0.167603   
DisparateImpactRemover(0.5)             0.011210          0.195905   
DisparateImpactRemover(0.7)             0.040937          0.125050   
DisparateImpactRemover(1.0)             0.040937          0.145122   
CalibratedEqOddsPostprocessing          0.042138          0.187676   
RejectOptionClassification(EO)          0.002702          0.178242   

Disparate Parity                Cross Validate 3  Cross Validate 4  \
Baseline                                   -0.08         -0.000300   
AdversarialDebiase                          0.25         -0.090682   
GridSearchReduction(EO)                    -0.09         -0.000901   
GridSearchReduction(DP)                    -0.08          0.009408   
Reweight                                   -0.03         -0.010610   
DisparateImpactRemover(0.5)                -0.07          0.009408   
DisparateImpactRemover(0.7)                -0.09          0.050646   
DisparateImpactRemover(1.0)                -0.05          0.029426   
CalibratedEqOddsPostprocessing             -0.06          0.029426   
RejectOptionClassification(EO)             -0.06         -0.011210   

Disparate Parity                Cross Validate 5   Average  
Baseline                                0.019256  0.027743  
AdversarialDebiase                     -0.937695 -0.061484  
GridSearchReduction(EO)                -0.002722  0.017584  
GridSearchReduction(DP)                -0.028430  0.011954  
Reweight                                0.035790  0.034919  
DisparateImpactRemover(0.5)             0.015627  0.032430  
DisparateImpactRemover(0.7)             0.024801  0.030287  
DisparateImpactRemover(1.0)             0.032161  0.039529  
CalibratedEqOddsPostprocessing          0.046779  0.049204  
RejectOptionClassification(EO)          0.066942  0.035335

Equal Opportuniy                Cross Validate 1  Cross Validate 2  \
Baseline                               -0.011574         -0.017857   
AdversarialDebiase                      0.055556          0.255952   
GridSearchReduction(EO)                -0.030093          0.000000   
GridSearchReduction(DP)                -0.030093          0.029762   
Reweight                               -0.009259         -0.017857   
DisparateImpactRemover(0.5)            -0.006944          0.029762   
DisparateImpactRemover(0.7)            -0.006944         -0.047619   
DisparateImpactRemover(1.0)            -0.006944          0.000000   
CalibratedEqOddsPostprocessing          0.030093          0.023810   
RejectOptionClassification(EO)         -0.030093          0.000000   

Equal Opportuniy                Cross Validate 3  Cross Validate 4  \
Baseline                                0.000760          0.098039   
AdversarialDebiase                      0.303571         -0.019608   
GridSearchReduction(EO)                -0.017097          0.098039   
GridSearchReduction(DP)                 0.000760          0.098039   
Reweight                                0.100304          0.078431   
DisparateImpactRemover(0.5)             0.043313          0.098039   
DisparateImpactRemover(0.7)             0.000760          0.156863   
DisparateImpactRemover(1.0)             0.057751          0.117647   
CalibratedEqOddsPostprocessing          0.022036          0.098039   
RejectOptionClassification(EO)          0.039894          0.117647   

Equal Opportuniy                Cross Validate 5   Average  
Baseline                                0.060823  0.026038  
AdversarialDebiase                     -0.906977 -0.062301  
GridSearchReduction(EO)                -0.020125  0.006145  
GridSearchReduction(DP)                -0.016100  0.016474  
Reweight                                0.060823  0.042488  
DisparateImpactRemover(0.5)             0.022361  0.037306  
DisparateImpactRemover(0.7)             0.060823  0.032776  
DisparateImpactRemover(1.0)             0.037567  0.041204  
CalibratedEqOddsPostprocessing          0.060823  0.046960  
RejectOptionClassification(EO)          0.107335  0.046957

Equalized Odds                  Cross Validate 1  Cross Validate 2  \
Baseline                               -0.088795          0.021382   
AdversarialDebiase                     -0.088795          0.337171   
GridSearchReduction(EO)                -0.101903          0.005757   
GridSearchReduction(DP)                -0.130233         -0.036184   
Reweight                               -0.112051          0.005757   
DisparateImpactRemover(0.5)            -0.112051          0.021382   
DisparateImpactRemover(0.7)            -0.052431         -0.036184   
DisparateImpactRemover(1.0)            -0.052431         -0.025493   
CalibratedEqOddsPostprocessing         -0.088795          0.005757   
RejectOptionClassification(EO)         -0.106977          0.005757   

Equalized Odds                  Cross Validate 3  Cross Validate 4  \
Baseline                               -0.015437         -0.053512   
AdversarialDebiase                      0.313036         -0.118729   
GridSearchReduction(EO)                -0.015437         -0.053512   
GridSearchReduction(DP)                -0.015437         -0.031773   
Reweight                               -0.015437         -0.053512   
DisparateImpactRemover(0.5)            -0.038165         -0.031773   
DisparateImpactRemover(0.7)            -0.042024         -0.010033   
DisparateImpactRemover(1.0)            -0.015437         -0.012542   
CalibratedEqOddsPostprocessing          0.003431          0.009197   
RejectOptionClassification(EO)         -0.015437         -0.096990   

Equalized Odds                  Cross Validate 5   Average  
Baseline                               -0.012061 -0.029685  
AdversarialDebiase                     -0.964912 -0.104446  
GridSearchReduction(EO)                 0.019737 -0.029072  
GridSearchReduction(DP)                -0.032895 -0.049304  
Reweight                                0.019737 -0.031101  
DisparateImpactRemover(0.5)             0.016447 -0.028832  
DisparateImpactRemover(0.7)            -0.001096 -0.028354  
DisparateImpactRemover(1.0)             0.033991 -0.014383  
CalibratedEqOddsPostprocessing          0.040570 -0.005968  
RejectOptionClassification(EO)          0.037281 -0.035273

In [52]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_DP_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Statistical parity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(DP)",RejectOptionClassification_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


Accuracy                        Cross Validate 1  Cross Validate 2  \
Baseline                                   0.895             0.940   
AdversarialDebiase                         0.860             0.840   
GridSearchReduction(EO)                    0.875             0.940   
GridSearchReduction(DP)                    0.890             0.925   
Reweight                                   0.890             0.935   
DisparateImpactRemover(0.5)                0.880             0.930   
DisparateImpactRemover(0.7)                0.885             0.915   
DisparateImpactRemover(1.0)                0.885             0.895   
CalibratedEqOddsPostprocessing             0.885             0.935   
RejectOptionClassification(EO)             0.885             0.940   
RejectOptionClassification(DP)             0.875             0.940   

Accuracy                        Cross Validate 3  Cross Validate 4  \
Baseline                                   0.905             0.895   
AdversarialDebiase                         0.810             0.870   
GridSearchReduction(EO)                    0.910             0.905   
GridSearchReduction(DP)                    0.905             0.910   
Reweight                                   0.910             0.900   
DisparateImpactRemover(0.5)                0.910             0.910   
DisparateImpactRemover(0.7)                0.890             0.900   
DisparateImpactRemover(1.0)                0.900             0.900   
CalibratedEqOddsPostprocessing             0.905             0.910   
RejectOptionClassification(EO)             0.905             0.890   
RejectOptionClassification(DP)             0.890             0.915   

Accuracy                        Cross Validate 5  Average  
Baseline                                   0.860    0.899  
AdversarialDebiase                         0.530    0.782  
GridSearchReduction(EO)                    0.890    0.904  
GridSearchReduction(DP)                    0.885    0.903  
Reweight                                   0.880    0.903  
DisparateImpactRemover(0.5)                0.900    0.906  
DisparateImpactRemover(0.7)                0.885    0.895  
DisparateImpactRemover(1.0)                0.890    0.894  
CalibratedEqOddsPostprocessing             0.875    0.902  
RejectOptionClassification(EO)             0.895    0.903  
RejectOptionClassification(DP)             0.885    0.901

Disparate Impact                Cross Validate 1  Cross Validate 2  \
Baseline                                1.042549          1.457706   
AdversarialDebiase                      1.104330          2.437200   
GridSearchReduction(EO)                 1.005969          1.449848   
GridSearchReduction(DP)                 0.986009          1.430202   
Reweight                                1.022528          1.422999   
DisparateImpactRemover(0.5)             1.021786          1.532460   
DisparateImpactRemover(0.7)             1.082676          1.315603   
DisparateImpactRemover(1.0)             1.082676          1.375195   
CalibratedEqOddsPostprocessing          1.081891          1.485210   
RejectOptionClassification(EO)          1.004971          1.449848   
RejectOptionClassification(DP)          1.004971          1.522340   

Disparate Impact                Cross Validate 3  Cross Validate 4  \
Baseline                                0.851852          0.999406   
AdversarialDebiase                      1.609756          0.837109   
GridSearchReduction(EO)                 0.836364          0.998252   
GridSearchReduction(DP)                 0.851852          1.018625   
Reweight                                0.942308          0.979417   
DisparateImpactRemover(0.5)             0.872727          1.018625   
DisparateImpactRemover(0.7)             0.839286          1.109169   
DisparateImpactRemover(1.0)             0.903846          1.059466   
CalibratedEqOddsPostprocessing          0.888889          1.059466   
RejectOptionClassification(EO)          0.886792          0.978679   
RejectOptionClassification(DP)          0.836364          1.039846   

Disparate Impact                Cross Validate 5   Average  
Baseline                                1.039602  1.078223  
AdversarialDebiase                      0.044778  1.206634  
GridSearchReduction(EO)                 0.994402  1.056967  
GridSearchReduction(DP)                 0.945633  1.046464  
Reweight                                1.078022  1.089055  
DisparateImpactRemover(0.5)             1.033398  1.095799  
DisparateImpactRemover(0.7)             1.054066  1.080160  
DisparateImpactRemover(1.0)             1.073031  1.098843  
CalibratedEqOddsPostprocessing          1.101978  1.123487  
RejectOptionClassification(EO)          1.148912  1.093840  
RejectOptionClassification(DP)          1.020350  1.084774

Disparate Parity                Cross Validate 1  Cross Validate 2  \
Baseline                                0.022720          0.177037   
AdversarialDebiase                      0.050646          0.420313   
GridSearchReduction(EO)                 0.003303          0.178242   
GridSearchReduction(DP)                -0.007607          0.166399   
Reweight                                0.011811          0.167603   
DisparateImpactRemover(0.5)             0.011210          0.195905   
DisparateImpactRemover(0.7)             0.040937          0.125050   
DisparateImpactRemover(1.0)             0.040937          0.145122   
CalibratedEqOddsPostprocessing          0.042138          0.187676   
RejectOptionClassification(EO)          0.002702          0.178242   
RejectOptionClassification(DP)          0.002702          0.197110   

Disparate Parity                Cross Validate 3  Cross Validate 4  \
Baseline                                   -0.08         -0.000300   
AdversarialDebiase                          0.25         -0.090682   
GridSearchReduction(EO)                    -0.09         -0.000901   
GridSearchReduction(DP)                    -0.08          0.009408   
Reweight                                   -0.03         -0.010610   
DisparateImpactRemover(0.5)                -0.07          0.009408   
DisparateImpactRemover(0.7)                -0.09          0.050646   
DisparateImpactRemover(1.0)                -0.05          0.029426   
CalibratedEqOddsPostprocessing             -0.06          0.029426   
RejectOptionClassification(EO)             -0.06         -0.011210   
RejectOptionClassification(DP)             -0.09          0.019718   

Disparate Parity                Cross Validate 5   Average  
Baseline                                0.019256  0.027743  
AdversarialDebiase                     -0.937695 -0.061484  
GridSearchReduction(EO)                -0.002722  0.017584  
GridSearchReduction(DP)                -0.028430  0.011954  
Reweight                                0.035790  0.034919  
DisparateImpactRemover(0.5)             0.015627  0.032430  
DisparateImpactRemover(0.7)             0.024801  0.030287  
DisparateImpactRemover(1.0)             0.032161  0.039529  
CalibratedEqOddsPostprocessing          0.046779  0.049204  
RejectOptionClassification(EO)          0.066942  0.035335  
RejectOptionClassification(DP)          0.010082  0.027922

Equal Opportuniy                Cross Validate 1  Cross Validate 2  \
Baseline                               -0.011574         -0.017857   
AdversarialDebiase                      0.055556          0.255952   
GridSearchReduction(EO)                -0.030093          0.000000   
GridSearchReduction(DP)                -0.030093          0.029762   
Reweight                               -0.009259         -0.017857   
DisparateImpactRemover(0.5)            -0.006944          0.029762   
DisparateImpactRemover(0.7)            -0.006944         -0.047619   
DisparateImpactRemover(1.0)            -0.006944          0.000000   
CalibratedEqOddsPostprocessing          0.030093          0.023810   
RejectOptionClassification(EO)         -0.030093          0.000000   
RejectOptionClassification(DP)         -0.009259          0.023810   

Equal Opportuniy                Cross Validate 3  Cross Validate 4  \
Baseline                                0.000760          0.098039   
AdversarialDebiase                      0.303571         -0.019608   
GridSearchReduction(EO)                -0.017097          0.098039   
GridSearchReduction(DP)                 0.000760          0.098039   
Reweight                                0.100304          0.078431   
DisparateImpactRemover(0.5)             0.043313          0.098039   
DisparateImpactRemover(0.7)             0.000760          0.156863   
DisparateImpactRemover(1.0)             0.057751          0.117647   
CalibratedEqOddsPostprocessing          0.022036          0.098039   
RejectOptionClassification(EO)          0.039894          0.117647   
RejectOptionClassification(DP)          0.018617          0.098039   

Equal Opportuniy                Cross Validate 5   Average  
Baseline                                0.060823  0.026038  
AdversarialDebiase                     -0.906977 -0.062301  
GridSearchReduction(EO)                -0.020125  0.006145  
GridSearchReduction(DP)                -0.016100  0.016474  
Reweight                                0.060823  0.042488  
DisparateImpactRemover(0.5)             0.022361  0.037306  
DisparateImpactRemover(0.7)             0.060823  0.032776  
DisparateImpactRemover(1.0)             0.037567  0.041204  
CalibratedEqOddsPostprocessing          0.060823  0.046960  
RejectOptionClassification(EO)          0.107335  0.046957  
RejectOptionClassification(DP)          0.045617  0.035365

Equalized Odds                  Cross Validate 1  Cross Validate 2  \
Baseline                               -0.088795          0.021382   
AdversarialDebiase                     -0.088795          0.337171   
GridSearchReduction(EO)                -0.101903          0.005757   
GridSearchReduction(DP)                -0.130233         -0.036184   
Reweight                               -0.112051          0.005757   
DisparateImpactRemover(0.5)            -0.112051          0.021382   
DisparateImpactRemover(0.7)            -0.052431         -0.036184   
DisparateImpactRemover(1.0)            -0.052431         -0.025493   
CalibratedEqOddsPostprocessing         -0.088795          0.005757   
RejectOptionClassification(EO)         -0.106977          0.005757   
RejectOptionClassification(DP)         -0.125159          0.021382   

Equalized Odds                  Cross Validate 3  Cross Validate 4  \
Baseline                               -0.015437         -0.053512   
AdversarialDebiase                      0.313036         -0.118729   
GridSearchReduction(EO)                -0.015437         -0.053512   
GridSearchReduction(DP)                -0.015437         -0.031773   
Reweight                               -0.015437         -0.053512   
DisparateImpactRemover(0.5)            -0.038165         -0.031773   
DisparateImpactRemover(0.7)            -0.042024         -0.010033   
DisparateImpactRemover(1.0)            -0.015437         -0.012542   
CalibratedEqOddsPostprocessing          0.003431          0.009197   
RejectOptionClassification(EO)         -0.015437         -0.096990   
RejectOptionClassification(DP)         -0.060892         -0.010033   

Equalized Odds                  Cross Validate 5   Average  
Baseline                               -0.012061 -0.029685  
AdversarialDebiase                     -0.964912 -0.104446  
GridSearchReduction(EO)                 0.019737 -0.029072  
GridSearchReduction(DP)                -0.032895 -0.049304  
Reweight                                0.019737 -0.031101  
DisparateImpactRemover(0.5)             0.016447 -0.028832  
DisparateImpactRemover(0.7)            -0.001096 -0.028354  
DisparateImpactRemover(1.0)             0.033991 -0.014383  
CalibratedEqOddsPostprocessing          0.040570 -0.005968  
RejectOptionClassification(EO)          0.037281 -0.035273  
RejectOptionClassification(DP)         -0.015351 -0.038011

In [53]:
Accuracy_data.to_csv("Benchmark_college_accuracy.csv")
DisparateImpact_data.to_csv("Benchmark_college_disparate_impact.csv")
DisparateParity_data.to_csv("Benchmark_college_disparate_parity.csv")
EqualOpportunity_data.to_csv("Benchmark_college_equal_opportunity.csv")
EqualizedOdds_data.to_csv("Benchmark_college_equalized_odds.csv")